In [ ]:
from coincidence_matching import match_parts, get_export_id_types
from zipfile import ZipFile
import json
import pandas as pd
from matplotlib import pyplot as plt
from tqdm import tqdm
from automate import Part
import numpy as np
import meshplot as mp
from collections import Counter
import torch

from brepmatching.data import BRepMatchingDataset

In [ ]:
import os
from pathlib import Path
str(Path.home().joinpath('.config','onshapecreds.json'))

In [ ]:
https://drive.google.com/file/d/1O7FajIOF5YT_1zW337pm6XE_B2gEhUUc
https://drive.google.com/file/d/1OK4qyuUfSXqA-wd64p_f2pG0rxeeqXGM
https://drive.google.com/file/d/1NnUBCkIUrYBTXvEC8IXGE4V2UNDTG6T6

In [70]:
def gdrive_wget(FILEID, FILENAME):
    return f"wget --load-cookies /tmp/cookies.txt \"https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id={FILEID}' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id={FILEID}\" -O {FILENAME} && rm -rf /tmp/cookies.txt"

In [73]:
print(gdrive_wget('1NnUBCkIUrYBTXvEC8IXGE4V2UNDTG6T6','Topo.zip'))

wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1NnUBCkIUrYBTXvEC8IXGE4V2UNDTG6T6' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/
/p')&id=1NnUBCkIUrYBTXvEC8IXGE4V2UNDTG6T6" -O Topo.zip && rm -rf /tmp/cookies.txt


In [ ]:
wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1O7FajIOF5YT_1zW337pm6XE_B2gEhUUc' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1O7FajIOF5YT_1zW337pm6XE_B2gEhUUc" -O TopoAndGeo.zip && rm -rf /tmp/cookies.txt

In [ ]:
from zipfile import ZipFile
import pandas as pd
from tqdm import tqdm
from apikey.onshape import Onshape
from pathlib import Path

api = Onshape(stack='https://cad.onshape.com', creds=str(Path.home().joinpath('.config','onshapecreds.json')), logging=False)

with ZipFile('../data/TopoandGeoV2FullRunWith100SamplesBaseline.zip','r') as zf:
    with zf.open('data/baseline/allVariationsWithBaseline.csv','r') as f:
        variations = pd.read_csv(f)
    variations[:100]
    variations = variations[variations.fail == 0]

    failures = []
    written = set()
    with ZipFile('../data/redownload.zip','w') as wzf:
        for i in tqdm(range(len(variations))):
            d = variations.iloc[i]

            response_orig = api.request(method='get', path=f'/api/partstudios/d/{d.did}/m/{d.mv_orig}/e/{d.eid}/parasolid', query={'includeExportIds':True})
            response_var = api.request(method='get', path=f'/api/partstudios/d/{d.did}/m/{d.mv_var}/e/{d.eid}/parasolid', query={'includeExportIds':True})

            if response_orig.status_code != 200 or response_var.status_code != 200:
                failures.append(d)
                continue
            
            orig_path = 'data/BrepsWithReference/' + d.ps_orig
            var_path = 'data/BrepsWithReference/' + d.ps_var

            if orig_path not in written:
                with wzf.open(orig_path, 'w') as f:
                    f.write(response_orig.text.encode('utf-8'))
                written.add(orig_path)
            if var_path not in written:
                with wzf.open(var_path, 'w') as f:
                    f.write(response_var.text.encode('utf-8'))
                written.add(var_path)

In [ ]:
#with ZipFile('../data/TopoandGeoV2FullRunWith100SamplesBaseline.zip','r') as zf:

with ZipFile('../data/GeoV2FullRunBaseline.zip','r') as zf:

    with zf.open('data/baseline/allVariationsWithBaseline.csv','r') as f:
        variations = pd.read_csv(f)

    data = variations[:100]
    data = data[(data.fail == 0) & (data.baselineNew != ' ') ]

    incomplete = []
    incompatible = []
    diff_geo = []

    m_f = []
    m_e = []
    m_v = []

    A = []
    B = []
    M = []

    for i in tqdm(range(len(data))):
        d = data.iloc[i]
        path_a = 'data/BrepsWithReference/' + d.ps_orig
        path_b = 'data/baseline/' + d.baselineOrig

        path_c = 'data/BrepsWithReference/' + d.ps_var
        path_d = 'data/baseline/' + d.baselineNew
        
        path_gt_match = 'data/Matches/' + d.matchFile
        path_bl_match = 'data/baseline/' + d.baselineMatch

        with zf.open(path_a,'r') as f:
            part_data_a = f.read().decode('utf-8')
        with zf.open(path_b,'r') as f:
            part_data_b = f.read().decode('utf-8')
        with zf.open(path_b,'r') as f:
            part_data_c = f.read().decode('utf-8')
        with zf.open(path_b,'r') as f:
            part_data_d = f.read().decode('utf-8')
        with zf.open(path_gt_match,'r') as f:
            gt_match = json.load(f)
        with zf.open(path_bl_match, 'r') as f:
            bl_match = json.load(f)

        p_a = Part(part_data_a)
        p_b = Part(part_data_b)
        p_c = Part(part_data_c)
        p_d = Part(part_data_d)

        parts_have_same_num = len(p_a.brep.nodes.faces) == len(p_b.brep.nodes.faces) and len(p_a.brep.nodes.edges) == len(p_b.brep.nodes.edges) and len(p_a.brep.nodes.vertices) == len(p_b.brep.nodes.vertices)

        matching = match_parts(part_data_a, part_data_b, True)
        complete_matching = len(matching.face_matches) == len(p_a.brep.nodes.faces) and len(matching.edge_matches) == len(p_a.brep.nodes.edges) and len(matching.vertex_matches) == len(p_a.brep.nodes.vertices)

        n_face_matches_missing = -len(matching.face_matches) + len(p_a.brep.nodes.faces)
        n_edge_matches_missing = -len(matching.edge_matches) + len(p_a.brep.nodes.edges)
        n_vertex_matches_missing =-len(matching.vertex_matches) + len(p_a.brep.nodes.vertices)

        m_f.append(n_face_matches_missing)
        m_e.append(n_edge_matches_missing)
        m_v.append(n_vertex_matches_missing)

        exact_geo = (p_a.mesh.V == p_b.mesh.V).all()

        if not parts_have_same_num:
            incompatible.append(i)
        if not complete_matching:
            incomplete.append(i)

        if not exact_geo:
            diff_geo.append(i)

        A.append(p_a)
        B.append(p_b)
        M.append(matching)


        # Exact matching of originals seems to work (and the tesselations are mostly identical!)
        # Let's see how well we match the versions with variations:

        var_matching = match_parts(part_data_c, part_data_d, True)

        id_classes_a = get_export_id_types(part_data_a)
        id_classes_b = get_export_id_types(part_data_b)
        id_classes_c = get_export_id_types(part_data_c)
        id_classes_d = get_export_id_types(part_data_d)

        good_classes = ['PK_CLASS_face', 'PK_CLASS_edge', 'PK_CLASS_vertex']

        #gt_topo_matches = [(v['val1'],v['val1']) for k,v in gt_match.items() if id_classes_a[v['val1']] in good_classes and id_classes_c[v['val2']] in good_classes]
        bl_topo_matches = [(v['val1'],v['val1']) for k,v in bl_match.items() if id_classes_b[v['val1']] in good_classes and id_classes_d[v['val2']] in good_classes]

        


In [ ]:
'JF4' in part_data_d

In [ ]:
data

In [ ]:
d.link_var

In [ ]:
part_b = Part(part_data_b)

In [ ]:
len([e.export_id for e in part_b.brep.nodes.edges])

In [ ]:
len([k for k,v in id_classes_b.items() if v == 'PK_CLASS_edge'])

In [ ]:
id_classes_b

In [ ]:
d.link_var

In [ ]:
data.baselineMatch

In [ ]:
from apikey.onshape import Onshape

In [ ]:
response = api.request(method='get', path=f'/api/partstudios/d/{d.did}/m/{d.mv_var}/e/{d.eid}/parasolid', query={'includeExportIds':True})#, query={'partIds':['JFD'],'includeExportIds':True})

In [ ]:
response.text == part_data_b

In [ ]:
d.mv_var

In [ ]:
d.did

In [ ]:
d.eid

In [ ]:
id_classes_b